In [0]:
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('viya01.onazure.sas.com',5570))
if result == 0:
   print("SUCCESS! Port is open") 
else:
    print("Failed. Port is NOT open")
sock.close()

SUCCESS! Port is open

In [0]:
import pandas as pd
import os
# set os environement variables pointing to the SAS Viya CA certs, and the OpenSSL libs
os.environ['CAS_CLIENT_SSL_CA_LIST']='/usr/local/share/ca-certificates/sas_ca.crt'
os.environ['TKESSL_OPENSSL_LIB']='/usr/lib/x86_64-linux-gnu/libssl.so.1.1'

#import package for SAS Scripting Wrapper for Analytics Transfer
import swat

In [0]:
# Create an http connection to SAS Viya CAS, specifying host name or url, using the swat package - the SAS Scripting Wrapper for Analytics Transfer  
viya=swat.CAS('viya01.onazure.sas.com', 5570, 'viyademo01', 'lnxsas')
# Set an active library for this session
viya.setsessopt(caslib='Public')

elapsed 0.000341s · user 0.000207s · sys 0.000108s · mem 0.256MB

In [0]:
# Load data to Viya either using a databricks client side load
testdata = viya.read_csv('/dbfs/FileStore/tables/CDN_ACCOUNTS/CDN_ACCOUNTS2.csv', casout=dict(name='CDN_Accounts2',caslib='Public', promote='true'))
# or load using a SAS Viya server side load
#
# use the swat CASTable object to treat a CAS Table like a pandas DataFrame
testdata = viya.CASTable('CDN_Accounts2')
testdata.head(20)

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,
1,217.0,Jean-Francois Tremblay,CP 123,Saint Marc QC,G0L 1TO,,90.0,,JFTremblay@bell.ca
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,QC,G1R 5P7,500.0,,
3,218.0,"Tremblay, JF",CP 123,St-Marc,Quebec,GOL 1T0,50.0,555-4545,
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,QC,G1R 5P7,400.0,info@cgi.ca,
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,Ontario,K1Z 1E9,100.0,7552313,ArnoldT@sas.com
6,204.0,Amar Singh,5264 Joel Avenue,"Burlington,ON",L7L3Y7,,300.0,,
7,220.0,C.G.I.,"140 Grand-Allee East, Unit 200",Quebec City,QC,G1R 5P7,600.0,627-2227,info@cgi.ca
8,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com
9,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa ON K1Z 1E9,,613 755-2313,90.0,,ArnoldT@sas.com


In [0]:
viya.dataStep.runCode( 
    code=''' data public.test_dq_from_Python ;
             set public.CDN_Accounts2 ;
                 City_Ident = dqIdentify(City,'Field Content','ENCAN');
                 Prov_Ident = dqIdentify(Prov,'Field Content','ENCAN');
                 Post_Ident = dqIdentify(PostCode,'Field Content','ENCAN');
                 Phone_Ident = dqIdentify(Phone,'Field Content','ENCAN');
                 Name_Ident = dqIdentify(Name,'Field Content','ENCAN');
             run;''')
# let's look at just the first six rows of our data ... 
dq = viya.CASTable('test_dq_from_Python')
dq.head(6)

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,City_Ident,Prov_Ident,Post_Ident,Phone_Ident,Name_Ident
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,,CITY,STATE/PROVINCE,POSTAL CODE,PHONE,INDIVIDUAL
1,217.0,Jean-Francois Tremblay,CP 123,Saint Marc QC,G0L 1TO,,90.0,,JFTremblay@bell.ca,INDIVIDUAL,POSTAL CODE,EMPTY,EMPTY,INDIVIDUAL
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,QC,G1R 5P7,500.0,,,STATE/PROVINCE,STATE/PROVINCE,POSTAL CODE,EMPTY,UNKNOWN
3,218.0,"Tremblay, JF",CP 123,St-Marc,Quebec,GOL 1T0,50.0,555-4545,,UNKNOWN,STATE/PROVINCE,POSTAL CODE,PHONE,UNKNOWN
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,QC,G1R 5P7,400.0,info@cgi.ca,,CITY,STATE/PROVINCE,POSTAL CODE,E-MAIL,ORGANIZATION
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,Ontario,K1Z 1E9,100.0,7552313,ArnoldT@sas.com,CITY,STATE/PROVINCE,POSTAL CODE,PHONE,INDIVIDUAL


In [0]:
# using the above results of the SAS QKB Identity Analysis,
# run dataStep code to move stray phone, email, Postal code info to their correct places AKA "Right-Fielding"
viya.dataStep.runCode( 
    code=''' data public.test_dq_from_Python;  
             set public.test_dq_from_Python;
                 if Phone_Ident = 'E-MAIL' then do;
                       Email = Phone;
                       Phone = '';
                       Phone_Ident = 'EMPTY';
                    end;
                 if Post_Ident = 'PHONE' then do;
                       Phone = PostCode;
                       PostCode = '';
                       Phone_Ident = 'PHONE';
                       Post_Ident = 'EMPTY';
                    end;
                 if Prov_Ident = 'POSTAL CODE' then do;
                       PostCode = Prov;
                       Prov     = '';
                       Post_Ident = 'POSTAL CODE';
                       Prov_Ident = 'EMPTY';
                    end;
              run;''')
dq.head(6)

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,City_Ident,Prov_Ident,Post_Ident,Phone_Ident,Name_Ident
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,,CITY,STATE/PROVINCE,POSTAL CODE,PHONE,INDIVIDUAL
1,217.0,Jean-Francois Tremblay,CP 123,Saint Marc QC,,G0L 1TO,90.0,,JFTremblay@bell.ca,INDIVIDUAL,EMPTY,POSTAL CODE,EMPTY,INDIVIDUAL
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,QC,G1R 5P7,500.0,,,STATE/PROVINCE,STATE/PROVINCE,POSTAL CODE,EMPTY,UNKNOWN
3,218.0,"Tremblay, JF",CP 123,St-Marc,Quebec,GOL 1T0,50.0,555-4545,,UNKNOWN,STATE/PROVINCE,POSTAL CODE,PHONE,UNKNOWN
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,QC,G1R 5P7,400.0,,info@cgi.ca,CITY,STATE/PROVINCE,POSTAL CODE,EMPTY,ORGANIZATION
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,Ontario,K1Z 1E9,100.0,7552313,ArnoldT@sas.com,CITY,STATE/PROVINCE,POSTAL CODE,PHONE,INDIVIDUAL


In [0]:
# run dataStep code to combine City, Province, PostCode fields for problem rows and parse out correct info
# using the SAS QKB parse definiton for City-State/Province-Postal Code.
viya.dataStep.runCode( 
    code=''' data public.test_dq_2(drop=City_Ident Prov_Ident Post_Ident Phone_Ident parsedCPP);  
             set public.test_dq_from_Python;
             if City_Ident ^= 'CITY' and (Prov_Ident='EMPTY' or Post_Ident='EMPTY') then do;
                 parsedCPP = dqParse(CATX(' ',City,Prov,PostCode), 'City - State/Province - Postal Code', 'ENCAN');
                 City     = dqParseTokenGet(parsedCPP, 'City', 'City - State/Province - Postal Code', 'ENCAN');
                 Prov     = dqParseTokenGet(parsedCPP, 'State/Province', 'City - State/Province - Postal Code', 'ENCAN');
                 PostCode = dqParseTokenGet(parsedCPP, 'Postal Code', 'City - State/Province - Postal Code', 'ENCAN');
                end;
             run;''')
dq2 = viya.CASTable('test_dq_2')
dq2.to_frame()

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,Name_Ident
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,,INDIVIDUAL
1,217.0,Jean-Francois Tremblay,CP 123,Saint Marc,QC,G0L 1TO,90.0,,JFTremblay@bell.ca,INDIVIDUAL
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,QC,G1R 5P7,500.0,,,UNKNOWN
3,218.0,"Tremblay, JF",CP 123,St-Marc,Quebec,GOL 1T0,50.0,555-4545,,UNKNOWN
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,QC,G1R 5P7,400.0,,info@cgi.ca,ORGANIZATION
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,Ontario,K1Z 1E9,100.0,7552313,ArnoldT@sas.com,INDIVIDUAL
6,204.0,Amar Singh,5264 Joel Avenue,Burlington,ON,L7L3Y7,300.0,,,INDIVIDUAL
7,220.0,C.G.I.,"140 Grand-Allee East, Unit 200",Quebec City,QC,G1R 5P7,600.0,627-2227,info@cgi.ca,ORGANIZATION
8,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com,INDIVIDUAL
9,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa,ON,K1Z 1E9,90.0,613 755-2313,ArnoldT@sas.com,INDIVIDUAL


In [0]:
# use the SAS QKB Standardize defintions for Province, PostCode, and Phone to standardize those columns in place
# use the SAS QKB Gender Analysis definition to enrich the data with a gender field, based on the Name 
viya.dataStep.runCode( 
    code=''' data public.test_dq_2(drop=Name_Ident);  
             set public.test_dq_2;
             Prov    = dqStandardize(Prov,'State/Province (Postal Standard)','ENCAN');
             Phone   = dqStandardize(Phone,'Phone');
             PostCode= dqStandardize(PostCode,'Postal Code','ENCAN');
             gender  = dqGender(Name,'Name','ENCAN') ;
             if Name_Ident = 'ORGANIZATION' then gender = '-';
            run;''')
dq2.to_frame()

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,gender
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819) 555 2334,,M
1,217.0,Jean-Francois Tremblay,CP 123,Saint Marc,QC,G0L 1T0,90.0,,JFTremblay@bell.ca,M
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,QC,G1R 5P7,500.0,,,U
3,218.0,"Tremblay, JF",CP 123,St-Marc,QC,G0L 1T0,50.0,555 4545,,U
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,QC,G1R 5P7,400.0,,info@cgi.ca,-
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,ON,K1Z 1E9,100.0,755 2313,ArnoldT@sas.com,U
6,204.0,Amar Singh,5264 Joel Avenue,Burlington,ON,L7L 3Y7,300.0,,,M
7,220.0,C.G.I.,"140 Grand-Allee East, Unit 200",Quebec City,QC,G1R 5P7,600.0,627 2227,info@cgi.ca,-
8,205.0,Jack Plant,,,,,50.0,555 2334,JPlante@gmail.com,M
9,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa,ON,K1Z 1E9,90.0,(613) 755 2313,ArnoldT@sas.com,M


In [0]:
# using the SAS QKB matchcode definitions create matchcodes on Name, Address, City.
viya.dataStep.runCode( 
    code=''' data public.test_dq_3;  
             set public.test_dq_2;
             Name_matchcode55 = dqMatch(Name,'Name',55,'ENCAN');
             Org_matchcode85 = dqMatch(Name,'Organization',85,'ENCAN');
             Address_matchcode70 = dqMatch(Address,'Address',70,'ENCAN');
             City_matchcode85 = dqMatch(City,'City',85,'ENCAN');
             if length(Phone) > 7 then Phone7 = substr(Phone,length(Phone)-7,8);
            run;''')
# let's look at results for rows 3 to 14 ...
dq3 = viya.CASTable('test_dq_3')
dq3[['ID','Name','Address','City','Name_matchcode55','Org_matchcode85','Address_matchcode70','City_matchcode85','Phone7']].fetch(from_=3,to=14)

,ID,Name,Address,City,Name_matchcode55,Org_matchcode85,Address_matchcode70,City_matchcode85,Phone7
2,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,3F7$$$$$$$$$$$$$$$$$$$$$$$$,JF7$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ZSFYP~$$$$$$$$...,3LM3$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,
3,218.0,"Tremblay, JF",CP 123,St-Marc,~YBM$$$$$$$$$$$C$$$$$$$$$$$,~Y£BMW£RCG$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$N@MZH$$$$$$$$$$$$$$$$$$...,4~BY3$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,555 4545
4,203.0,CGI Ltée,"140 Grand-Allee Est, Bureau 200",Ville de Québec,W~7$$$$$$$$$$$$J$$$$$$$$$$$,JF7$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ZSFYP~$$$$$$$$...,3LM3$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,
5,219.0,A. Toporowski,"38 Metropole Private, Unit 1605",Ottawa,~MYL$$$$$$$$$$$&$$$$$$$$$$$,~£N£Y£L437$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$KB~YNWNV~$$$$$...,@~L$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,755 2313
6,204.0,Amar Singh,5264 Joel Avenue,Burlington,4BF$$$$$$$$$$$$&$$$$$$$$$$$,&B&Y47PF$$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$5H6CW$$$$$$$$$...,MYWPF~P$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,
7,220.0,C.G.I.,"140 Grand-Allee East, Unit 200",Quebec City,7$$$$$$$$$$$$$$J$$$$$$$$$$$,JF7$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ZSFYP~$$$$$$$$...,3LM3$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,627 2227
8,205.0,Jack Plant,,,NWB~$$$$$$$$$$$C$$$$$$$$$$$,C&J3NW£P~$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$...,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,555 2334
9,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa,~MYL$$$$$$$$$$$&$$$$$$$$$$$,BY&YP7~£N£Y£L437$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$KB~YNW$$$$$$$$...,@~L$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,755 2313
10,207.0,Ms MJ Belanger,4500 Sherbrooke St W,Montreal,MWBF$$$$$$$$$$$B$$$$$$$$$$$,B4BCM£W£PF£Y$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$S5042YMY3$$$$$...,BP~YW$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,799 9239
11,208.0,CGI Canada Inc,200-140 Grand Allee,Quebec,&B3$$$$$$$$$$$$J$$$$$$$$$$$,JF7$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ZSFYP~$$$$$$$$...,3LM3$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,627 2227


In [0]:
#load Entity Resolution CAS action set 
viya.loadactionset(actionset="entityRes")

§ actionset 
 
 entityRes 
 
 
 elapsed 0.00141s · user 0.000234s · sys 0.00109s · mem 0.205MB

In [0]:
# use the entityres.match CAS action to match rows on: (Name & Address & Postal Code) 
#   OR  (Name & City & Province)  OR  (Name & Phone)  OR  (Name & Email) OR (Org & Postcode)
#
dq_clustered = viya.CASTable('test_Clustered', groupBy='CLUSTERID', replace=True)
viya.entityres.match(clusterid='CLUSTERID',  
        intable=dq3,
        matchrules=[{'rule':[{'columns':['Name_matchcode55','Address_matchcode70','Postcode']},]},
                    {'rule':[{'columns':['Name_matchcode55','City_matchcode85','Prov']}]},
                    {'rule':[{'columns':['Name_matchcode55','Phone7']}]},
                    {'rule':[{'columns':['Name_matchcode55','Email']}]},
                    {'rule':[{'columns':['Org_matchcode85','Postcode']}]}
                   ],
        outtable=dq_clustered) 

# display cluster results.  CLUSTERID is a 24-byte character string 
dq_clustered[['CLUSTERID','ID','Name','Address','City','Phone','Email']].sort_values('CLUSTERID').to_frame()

,CLUSTERID,ID,Name,Address,City,Phone,Email
0,0.0,215.0,Jacques Plante,14 Denise Unit 201,Cantley,,
1,0.0,212.0,Jacques Plante,"14, Chemin Denis, app 201",Cantley,,
2,0.0,205.0,Jack Plant,,,555 2334,JPlante@gmail.com
3,0.0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,(819) 555 2334,
4,0.0,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,555 2334,
5,1.0,216.0,JF Tremblay,P.O. Box 123,St-Marc-du-Lac-Long,(819) 555 4545,
6,1.0,217.0,Jean-Francois Tremblay,CP 123,Saint Marc,,JFTremblay@bell.ca
7,1.0,218.0,"Tremblay, JF",CP 123,St-Marc,555 4545,
8,2.0,202.0,CGI,140 Rue Grand-Allee Est Unit 200,Quebec,,
9,2.0,208.0,CGI Canada Inc,200-140 Grand Allee,Quebec,(418) 627 2227,


In [0]:
# run datastep code to create a "golden record" for each cluster with best info, and total amount.  AKA "Survivorship"
viya.dataStep.runCode( 
    code=''' data public.test_done
                     (drop= max_Name min_gender max_Address max_City max_Postal tot_Amount max_Phone max_Email
                            ID Name_matchcode55 Address_matchcode70 City_matchcode85 Phone7); 
             set public.test_Clustered;
             by CLUSTERID;
             
             retain   max_Name  min_gender  max_Address  max_City  max_Postal  tot_Amount  max_Phone  max_Email;
             if first.CLUSTERID then do;
                      max_Name = Name;
                      min_gender = gender;
                      max_Address = Address;
                      max_City = City;
                      max_Postal = PostCode;
                      tot_Amount = Amount;
                      max_Phone = Phone;
                      max_Email = Email;
                      end;
                 else do;
                       if length(Name) < length(max_Name)       then Name = max_Name;       else max_Name = Name;
                       if gender > min_gender                   then gender = min_gender;   else min_gender = gender; 
                       if length(Address) < length(max_Address) then Address = max_Address; else max_Address = Address;
                       if length(City) < length(max_City)       then City = max_City;       else max_City = City;
                       if length(PostCode) < length(max_Postal) then PostCode = max_Postal; else max_Postal = PostCode;
                       tot_Amount = Amount + tot_Amount;
                       Amount = tot_Amount;
                       if length(Phone) < length(max_Phone)     then Phone = max_Phone;     else max_Phone = Phone;
                       if length(Email) < length(max_Email)     then Email = max_Email;     else max_Email = Email;
                       end;
             if last.CLUSTERID then output;
    run;''')
dq_done = viya.CASTable('test_done')
dq_done[['CLUSTERID','Name','gender','Address','City','Prov','PostCode','Amount','Phone','Email']].sort_values('CLUSTERID').head(20)

,CLUSTERID,Name,gender,Address,City,Prov,PostCode,Amount,Phone,Email
0,0.0,Mr. Jacques Plante,M,"14, Chemin Denis, app 201",Gatineau,QC,J8V 3J5,300.0,(819) 555 2334,JPlante@gmail.com
1,1.0,Jean-Francois Tremblay,M,P.O. Box 123,St-Marc-du-Lac-Long,QC,G0L 1T0,340.0,(819) 555 4545,JFTremblay@bell.ca
2,2.0,CGI Canada Inc,-,140 Rue Grand-Allee Est Unit 200,Ville de Québec,QC,G1R 5P7,2000.0,(418) 627 2227,info@cgi.ca
3,3.0,Mr. Arnold Toporowski,M,"38 Metropole Private, Unit 1605",Ottawa,ON,K1Z 1E9,590.0,(613) 755 2313,ArnoldT@sas.com
4,4.0,Amar Singh,M,5264 Joel Avenue,Burlington,ON,L7L 3Y7,400.0,(905) 637 5119,amar.singh@lost.com
5,5.0,Mme Marie-Josée Bélanger,F,"4500, rue Sherbrook Ouest",Montréal,QC,H3Z 1E6,1950.0,(514) 799 9239,MJBelanger@bell.ca
